# Import Packages

In [72]:
from configs import get_config
from data_loader import get_loader

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.autograd import Variable

from tqdm import tqdm

# Set Configuration

In [77]:
config = get_config(
    parse=False,
    vocab_size=20000,
    hidden_size=100,
    n_channel_per_window=2,
    label_size=2,
    dropout=0.5)

In [78]:
config

Configurations
{'batch_size': 100,
 'data_dir': PosixPath('/Users/jmin/workspace/fastcampus_chatbot/Day_02/CNN/datasets'),
 'dropout': 0.5,
 'epochs': 20,
 'hidden_size': 100,
 'label_size': 2,
 'log_every_epoch': 1,
 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>,
 'lr': 0.001,
 'n_channel_per_window': 2,
 'optimizer': <class 'torch.optim.sgd.SGD'>,
 'save_dir': PosixPath('/Users/jmin/workspace/fastcampus_chatbot/Day_02/CNN/log'),
 'save_every_epoch': 1,
 'vocab_size': 20000}

# Load training data loader

In [80]:
train_loader = get_loader(batch_size=20, max_size=config.vocab_size, is_train=True, data_dir='./datasets/')

Building Vocabulary 



In [81]:
batch = next(iter(train_loader))
batch

In [84]:
# [max_seq_len, batch_size]
batch.text

Variable containing:
  3249    127   2623  ...     524    213   3657
    20     12    159  ...      12     13      9
     6     27    152  ...     376    785    173
        ...            ⋱           ...         
     1      1      1  ...       1      1      1
     1      1      1  ...       1      1      1
     1      1      1  ...       1      1      1
[torch.LongTensor of size 71x20]

In [38]:
# [batch_size]
batch.label

Variable containing:
 0
 1
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 0
[torch.LongTensor of size 20]

## Model

<img src="../images/cnn_text_classification.png", width=600, height=60>

In [35]:
class CNN(nn.Module):
    def __init__(self, config):
        super(CNN, self).__init__()
        self.config = config
        
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size)
        
        self.conv = nn.ModuleList([
            nn.Conv2d(
                in_channels=1,
                out_channels=config.n_channel_per_window,
                kernel_size=(3, config.hidden_size)),
            
            nn.Conv2d(
                in_channels=1,
                out_channels=config.n_channel_per_window,
                kernel_size=(4, config.hidden_size)),

            nn.Conv2d(
                in_channels=1,
                out_channels=config.n_channel_per_window,
                kernel_size=(5, config.hidden_size))
        ])
        
        n_total_channels = len(self.conv) * config.n_channel_per_window
        
        self.dropout = nn.Dropout(config.dropout)
        self.fc = nn.Linear(n_total_channels, config.label_size)
        
    def forward(self, x):
        """
        Args:
            x: [batch_size, max_seq_len]
        Return:
            logit: [batch_size, label_size]
        """
        
        # [batch_size, max_seq_len, hidden_size]
        x = self.embedding(x)
        
        # [batch_size, 1, max_seq_len, hidden_size]
        x = x.unsqueeze(1)
        
        # Apply Convolution filter followed by Max-pool
        out_list = []
        for conv in self.conv:
            
            ########## Convolution #########
            
            # [batch_size, n_kernels, _, 1]
            x_ = F.relu(conv(x))
            
            # [batch_size, n_kernels, _]
            x_ = x_.squeeze(3)
            
            ########## Max-pool #########
            
            # [batch_size, n_kernels, 1]
            x_ = F.max_pool1d(x_, x_.size(2))
            
            # [batch_size, n_kernels]
            x_ = x_.squeeze(2)
            
            out_list.append(x_)
        
        # [batch_size, 3 x n_kernels]
        out = torch.cat(out_list, 1)
        
        ######## Dropout ########
        out = self.dropout(out)
        
        # [batch_size, label_size]
        logit = self.fc(out)
        
        return logit

In [85]:
model = CNN(config)

In [86]:
model

CNN (
  (embedding): Embedding(20000, 100)
  (conv): ModuleList (
    (0): Conv2d(1, 2, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 2, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 2, kernel_size=(5, 100), stride=(1, 1))
  )
  (dropout): Dropout (p = 0.5)
  (fc): Linear (6 -> 2)
)

# Build loss function

In [87]:
loss_fn = config.loss_fn()

loss_fn

CrossEntropyLoss (
)

# Build Optimizer 

In [88]:
optimizer = config.optimizer(model.parameters(), config.lr)
optimizer

In [89]:
for epoch in range(2): # n_epochs
    print(f'Epoch: {epoch}')
    for batch_i, batch in enumerate(tqdm(train_loader)):
        # text: [max_seq_len, batch_size]
        # label: [batch_size]
        text, label = batch.text, batch.label

        # [batch_size, max_seq_len]
        text.data.t_()
        
        # [batch_size, 2]
        logit = model(text)
        
        # Calculate loss
        batch_loss = loss_fn(logit, label)
        batch_loss.backward()
        optimizer.step()
        
        if (batch_i + 1) % 50 == 0:
            tqdm.write(f'batch loss: {batch_loss.data}')


  0%|          | 0/7302 [00:00<?, ?it/s]

Epoch: 0



  1%|          | 49/7302 [00:02<04:43, 25.57it/s]
          
  1%|          | 52/7302 [00:02<05:09, 23.42it/s]

batch loss: 
 0.6101
[torch.FloatTensor of size 1]




  1%|▏         | 98/7302 [00:04<04:32, 26.45it/s]
          
  1%|▏         | 104/7302 [00:04<04:58, 24.08it/s]

...

  8%|▊         | 604/7302 [00:23<04:27, 25.04it/s]

batch loss: 
 0.8771
[torch.FloatTensor of size 1]



  9%|▉         | 640/7302 [00:24<03:53, 28.48it/s]

KeyboardInterrupt: 

  9%|▉         | 640/7302 [00:38<06:44, 16.46it/s]